In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load CMp125 output

In [5]:
import numpy as np
import xarray as xr
from xgcm import Grid

In [6]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")

    ## Load CM4p125's state variable output
    zdiag_path = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/"
    thetao = xr.open_mfdataset(f"{zdiag_path}/ts/annual/10yr/*thetao.nc", chunks={'time':1})
    volcello = xr.open_mfdataset(f"{zdiag_path}/ts/annual/10yr/*.volcello.nc", chunks={'time':1})
    state = xr.merge([thetao, volcello])

In [7]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    
    # get correct geo coords from super grid
    sg = xr.open_dataset("/archive/Raphael.Dussin/datasets/OM4p125/mosaic_c192_om4p125_bedmachine_v20210310_hydrographyKDunne20210614_unpacked/ocean_hgrid.nc")
    ocean_grid = xr.open_dataset(f"{zdiag_path}ocean_annual_z.static.nc")
    ocean_grid['deptho'] = ocean_grid['deptho'].where(~np.isnan(ocean_grid['deptho']), 0.)
    ocean_grid = ocean_grid.assign_coords({'geolon'  : xr.DataArray(sg['x'][1::2,1::2].data, dims=["yh", "xh"]),
                                           'geolat'  : xr.DataArray(sg['y'][1::2,1::2].data, dims=["yh", "xh"])})
    
    # correct state variable grids
    grid = Grid(state, coords={'X': {'center': 'xh',},
                            'Y': {'center': 'yh',},
                            'Z': {'center': 'z_l', 'outer': 'z_i'} }, periodic=['X'])
    state['geolon'] = xr.DataArray(data=ocean_grid['geolon'].data, dims=('yh', 'xh'))
    state['geolat'] = xr.DataArray(data=ocean_grid['geolat'].data, dims=('yh', 'xh'))
    state = state.assign_coords({'geolon': state['geolon'],
                                 'geolat': state['geolat']})

In [8]:
rho0 = 1035.
Cp = 3992.
heat_content = rho0*Cp*state['thetao']*state['volcello']

In [9]:
integrated_heat_content = heat_content.sel(z_l=slice(4500, None)).sum('z_l').compute()

In [10]:
integrated_heat_content.to_netcdf("/work/hfd/CM4-highres-WMT/data/abyssal_heat_content_annual.nc", mode="w")